# Session 5: Building a model from scratch (II)

Previously...

<p style="font-size:120%;font-style:italic">• Given an ECG signal, develop a method to classify each beat by its origin in the cardiac tissue</p>

Then we explored, understood, cleaned and homogeneized the data.

And when we tried a first simple machine learning model:

<table>
    <tr>
        <td style="padding-right: 30pt;">
            <table>
                <thead>
                    <tr>
                        <th></th>
                        <th>Pred 0</th>
                        <th>Pred 1</th>
                        <th>Pred 2</th>
                        <th>Pred 3</th>
                    </tr>
                </thead>
                <tbody>
                    <tr>
                        <td>True 0</td>
                        <td>2920</td>
                        <td>0</td>
                        <td>6</td>
                        <td>0</td>
                    </tr>
                    <tr>
                        <td>True 1</td>
                        <td>51</td>
                        <td>57</td>
                        <td>0</td>
                        <td>0</td>
                    </tr>
                    <tr>
                        <td>True 2</td>
                        <td>44</td>
                        <td>0</td>
                        <td>182</td>
                        <td>3</td>
                    </tr>
                    <tr>
                        <td>True 3</td>
                        <td>6</td>
                        <td>0</td>
                        <td>3</td>
                        <td>10</td>
                    </tr>
                </tbody>
            </table>
        </td>
        <td>
            <table>
                <thead>
                    <tr>
                        <th></th>
                        <th>Precision</th>
                        <th>Recall</th>
                        <th>F1-Score</th>
                        <th>Support</th>
                    </tr>
                </thead>
                <tbody>
                    <tr>
                        <td>0</td>
                        <td>0.97</td>
                        <td>1.00</td>
                        <td>0.98</td>
                        <td>2926.00</td>
                    </tr>
                    <tr>
                        <td>1</td>
                        <td>1.00</td>
                        <td>0.53</td>
                        <td>0.69</td>
                        <td>108.00</td>
                    </tr>
                    <tr>
                        <td>2</td>
                        <td>0.95</td>
                        <td>0.79</td>
                        <td>0.87</td>
                        <td>229.00</td>
                    </tr>
                    <tr>
                        <td>3</td>
                        <td>0.77</td>
                        <td>0.53</td>
                        <td>0.62</td>
                        <td>19.00</td>
                    </tr>
                    <tr>
                        <td>Accuracy</td>
                        <td>0.97</td>
                        <td>0.97</td>
                        <td>0.97</td>
                        <td>0.97</td>
                    </tr>
                    <tr>
                        <td>Macro Avg</td>
                        <td>0.92</td>
                        <td>0.71</td>
                        <td>0.79</td>
                        <td>3282.00</td>
                    </tr>
                    <tr>
                        <td>Weighted Avg</td>
                        <td>0.97</td>
                        <td>0.97</td>
                        <td>0.96</td>
                        <td>3282.00</td>
                    </tr>
                </tbody>
            </table>
        </td>
    </tr>
</table>

<p style="font-size:400%;color:darkred;text-align:center">🥳</p>

---- 
<p style="font-size:400%;color:darkred;text-align:center">⚠️⚠️ WRONG!!! ⚠️⚠️</p>

The last, quick experiment with a simple ML model is **fundamentally wrong**, and any conclusion we may get from it will be misleading!

### **What happened?**

 - After we segmented and homogeneized the raw signal for each individual heartbeat, we created a big `X` matrix with one row per beat, and proceeded in the standard way:

<pre>
<code class="language-python">
# Prepare the X and y matrices
X = np.hstack(df_sample2.beat_sig).T
X = X - np.median(X, axis=1, keepdims=True)
y = np.repeat(df_sample2.type, 2)
# Split in training and test
<span style="background-color: #ffff99;">X_train, X_test, y_train, y_test = <a href="https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html">train_test_split</a>(X, y, test_size=0.2, random_state=42)</span>
# Create the model, train it and test it
classifier = RandomForestClassifier()
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
</code>
</pre>

However, the core of any Machine Learning methodology is the assumption that **`X_train` and `X_test` are independent and indentically distributed**. We have violated the independence assumption in two different ways:
 - By including beats **from the same subject** in training and testing.
 - By including **the same beat** (with different projections corresponding to the leads) in training and testing.

This problem, commonly know as **data leakage**, is pervasive in real studies using ML, and the main cause of overestimated results, even in peer-reviewed publications.
 - Example: [Ankışhan et.al: *Early stage lung cancer detection from speech sounds in natural environments* (2024)](https://tomas-teijeiro.github.io/Slides_BCAM/#/5/5).

Let's recall the procedure for addressing any realistic problem with Machine Learning:
 1. Learning about the problem to be solved.
 2. Understanding the available data.
 3. Data cleaning, preparation and homogeneization.
 4. **Design of experimental validation.**
 5. Choose a model or set of models to evaluate.
 6. Training and evaluating the model(s).
 7. Extract conclusions, assess the limitations of the model.

The problem with step 4 is that it is totally dependent on the **specific problem** to be solved, and using the same strategy for two problems that in principle look similar may be catastrophic. 
 - For example, mixing data from the same patient in training and testing can be perfectly fine, and even required, if we are building a personalized model.

The following code cells load and prepare the data as we did it in the last session:

In [1]:
import os
os.environ["KERAS_BACKEND"] = "jax"
#os.environ["JAX_PLATFORMS"] = "cpu"
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"]="false"

#General imports
import collections
import numpy as np
import pandas as pd
import scipy.stats
import matplotlib.pyplot as plt
import seaborn as sns
import wfdb
import keras
#Figure settings to avoid super large plots
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['figure.dpi'] = 90
#Local path to the MIT-BIH Arrhythmia
MITDB = '/opt/tljh/common/mitdb'

In [2]:
#Global structures with target labels, mapping to classes, and list of recordings to analyze
BEAT_CODES = set(['N','L','R','B','A','a','J','S','V','F','e','j','n','E','/','f','Q'])
LABEL_MAP = {'N':0,'L':0,'R':0,'B':0,
             'A':1,'a':1,'J':1,'S':1,'e':1,'j':1,'n':1,
             'V':2,'E':2,
             'F':3,
             '/':4,'f':4,'Q':4}
REC_LIST = [r.strip() for r in open(f'{MITDB}/RECORDS', 'r').readlines()]

In Session 4, we agreed that we should remove leads `V5`, `V2` and `V4` due to the extremely small number of samples compared with the other leads. However, if we are doing the split at patient level, we need to consider all existing **lead combinations**. Let's get a quick summary:

In [3]:
collections.Counter([tuple(wfdb.rdrecord(f'{MITDB}/{r}').sig_name) for r in REC_LIST])

Counter({('MLII', 'V1'): 40,
         ('MLII', 'V5'): 2,
         ('V5', 'V2'): 2,
         ('MLII', 'V2'): 2,
         ('V5', 'MLII'): 1,
         ('MLII', 'V4'): 1})

It seems evident we should select the recordings with leads `(MLII, V1)`. Let's filter them:

In [4]:
REC_LIST = [r for r in REC_LIST if tuple(wfdb.rdrecord(f'{MITDB}/{r}').sig_name)==('MLII', 'V1')]
print(REC_LIST, len(REC_LIST))

['101', '105', '106', '107', '108', '109', '111', '112', '113', '115', '116', '118', '119', '121', '122', '200', '201', '202', '203', '205', '207', '208', '209', '210', '212', '213', '214', '215', '217', '219', '220', '221', '222', '223', '228', '230', '231', '232', '233', '234'] 40


In [5]:
#Code from the last session to load and organize every beat in a global dataframe
def adjust_array_length(arr, peak):
    """Utility function to adjust the length of each array"""
    desired_length = 1501
    target_peak_index = 750
    current_length = len(arr)

    start_index = max(0, peak - target_peak_index)
    end_index = min(current_length, peak + (desired_length - target_peak_index))

    # Truncate the array around the peak
    truncated_array = arr[start_index:end_index, :]

    left_padding = target_peak_index - (peak - start_index)
    right_padding = desired_length - len(truncated_array) - left_padding

    # Extend the array if needed
    if left_padding > 0:
        left_pad_value = truncated_array[0, :]
        left_extension = np.full((left_padding, 2), left_pad_value)
    else:
        left_extension = np.empty(shape=(0, 2))

    if right_padding > 0:
        right_pad_value = truncated_array[-1, :]
        right_extension = np.full((right_padding, 2), right_pad_value)
    else:
        right_extension = np.empty(shape=(0, 2))

    # Combine the extensions and truncated array
    adjusted_array = np.concatenate((left_extension, truncated_array, right_extension))

    return adjusted_array
    
# Creation of a Pandas dataframe with homogeneized data for every beat
full_data = []
offset = 18 #Window cut with respect to previous and next peak.
for r in REC_LIST:
    rec = wfdb.rdrecord(f'{MITDB}/{r}')
    anns = wfdb.rdann(f'{MITDB}/{r}', 'atr')
    beat_mask = np.array([s in BEAT_CODES for s in anns.symbol])
    beat_indices = np.where(beat_mask)[0]
    beat_types = np.array(anns.symbol)[beat_indices]
    beat_locations = anns.sample[beat_indices]
    for i in range(1, len(beat_indices)-1):
        beat_data = {}
        beat_data['beat_sig'] = adjust_array_length(rec.p_signal[beat_locations[i-1]
                                                    + offset:beat_locations[i+1]-offset,:],
                                                    beat_locations[i]-beat_locations[i-1])
        beat_data['type'] = LABEL_MAP[beat_types[i]]
        beat_data['leads'] =  rec.sig_name
        beat_data['recname'] = rec.record_name
        full_data.append(beat_data)
df_beats = pd.DataFrame(full_data)

We also concluded that class 4 should be removed from the analysis, basically for the very same reason that we removed some of the leads:

In [6]:
df_beats = df_beats.query('type!=4')

As a general recommendation, while you are developing and exploring initial models, do it with a restricted amount of data to speed-up things:

In [15]:
df_sample = df_beats.sample(n=10000, random_state=42)

#### 📋 Exercise 1

Define, build, train and validate a Machine Learning model to solve the heartbeat classification problem, considering all the issues we overlooked in Session 4.

 - The main metric to assess the quality of the model will be **weighted average F1 score** for the 4 classes.

__Hints:__
 - A CNN is probably the most promising model a priori. You may use a 2D CNN in case you want to input the two leads of each beat simultaneously, or a 1D CNN in case you want to use single leads.
     - Suggested architecture to get started:
         - 3 1D-Convolutional layers, with filter size 64, 128 and 256 (kernel size 3), each one followed by BatchNormalization and MaxPooling1D (with pool size 2).
         - 2 Dense layers (after a Flatten one), with 128 and 64 units. Dropout of 0.5 after each of them.
         - Output Dense layer, with 4 outputs (one per class).
 - Be extremely careful with the data splitting strategy. Make sure statistical independence is maintained between training, validation and test.
 - Start from a simple model, get a working pipeline, and optimize over that. Even the one tested in Session 4 could do the job for this.
 - Class imbalance may be a big problem when training. Consider playing with the `class_weight` and `sample_weight` parameters of `model.fit()`.
     - Accuracy is usually not a good metric to follow in imbalanced scenarios.
 - Once you have a model you are happy with, train it also to work with a **single lead** and compare the results.
 - Rhythm information (distance to the next and previous beats) is essential for a proper detection of some classes. While this information is already in the data format we developed, it may be difficult to extract automatically. Explicitly providing these values as features can help.
 - The ideal validation pipeline for this kind of problems with limited amount of subjects is leave-one-subject out cross-validation, or at least k-fold cross-validation.

In order to guarantee the independence of training and test sets, we will make the split ensuring no data from the same recording are in training and testing. The [`sklearn.model_selection.GroupShuffleSplit`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GroupShuffleSplit.html) splitter is specifically designed for these use cases:

In [45]:
#We get 80% of the beats for training and 20% for testing, ensuring recordings are different in
#both sets.
from sklearn.model_selection import GroupShuffleSplit
gsplit = GroupShuffleSplit(n_splits=1, test_size=0.2)
train_index, test_index = next(gsplit.split(df_sample.beat_sig, df_sample.type, df_sample.recname))
#Check that the split was done indeed in a proper way
print(set(df_sample.iloc[train_index].recname.unique()).isdisjoint(set(df_sample.iloc[test_index].recname.unique())))

True


In [46]:
#Convert the data to the classical X, y training and testing matrices.
X_train = np.stack(df_sample.iloc[train_index].beat_sig.values)
y_train = df_sample.iloc[train_index].type.values
y_train = np.array(y_train).reshape(-1, 1)
X_test = np.stack(df_sample.iloc[test_index].beat_sig.values)
y_test = df_sample.iloc[test_index].type.values

In [47]:
X_train.shape

(7739, 1501, 2)